In [1]:
import socket
import ssl
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.serialization import load_pem_private_key
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
import os
import tkinter as tk



In [2]:
encryption_choice = 0
msg = ""

def generate_rsa_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    return private_key

def generate_dh_keypair():
    parameters = dh.generate_parameters(generator=2, key_size=2048)
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()
    return private_key, public_key

# def derive_dh_shared_key(private_key, public_key):
#     shared_key = private_key.exchange(public_key)
#     return shared_key

def derive_dh_shared_key(private_key, public_key):
    shared_key = public_key.public_numbers().y ** private_key.private_numbers().x % public_key.public_numbers().parameter_numbers.g
    return shared_key.to_bytes((shared_key.bit_length() + 7) // 8, byteorder='big')


def rsa_encrypt(public_key, plaintext):
    ciphertext = public_key.encrypt(
        plaintext.encode(),
        padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)
        )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP( mgf=padding.MGF1(algorithm=hashes.SHA256()),
    algorithm=hashes.SHA256(),
    label=None))
    return plaintext.decode()

def aes_encrypt(key, plaintext):
    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return iv + ciphertext

def aes_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext.decode()

def start_client(msg_for_server,append_log):
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.load_verify_locations(cafile="server.crt")
    context.check_hostname = False
    client_socket.connect(("localhost", 8443))
    append_log("Connected to server.")
    
    with context.wrap_socket(client_socket, server_hostname="localhost") as ssock:
        
        append_log("SSL/TLS connection established.")
        
        # encryption_choice = input("Enter encryption method (1 for RSA, 2 for AES): ")
        # ssock.send(encryption_choice.encode())
        encryption_choice = ssock.recv(2048).decode()
        append_log(f"Encryption choice received from server: {encryption_choice}")
    
        if encryption_choice == "1":
            # RSA encryption and PKI key exchange
            private_key = generate_rsa_keypair()
            public_key = private_key.public_key()
            append_log(f"Public key generated. {public_key}")
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            
            #receive public key from server
            received_public_key = ssock.recv(2048)
            server_public_key = load_pem_public_key(received_public_key, backend=default_backend())
            
            
            # while True:
            msg_from_server = ssock.recv(2048)
            msg_from_server = rsa_decrypt(private_key, msg_from_server)
            append_log(f"Decrypted RSA message from server: {msg_from_server}")
            #send msg to server
            # msg_for_server = input("Enter your message for server: ")
        
            #encrypt msg with server's public key
            msg_for_server = rsa_encrypt(server_public_key, msg_for_server)
            append_log(f"Encrypted RSA message for server: {msg_for_server}")
            #send encrypted msg to server
            ssock.send(msg_for_server)
            append_log("Message sent to server.")
            
            #-----------------------------------------------
            #receive msg from server
      
                # if msg_from_server == "exit":
                #     break

        elif encryption_choice == "2":
            append_log("Diffie-Hellman key exchange initiated.")
            # AES encryption and Diffie-Hellman key exchange
            private_key, public_key = generate_dh_keypair()
            append_log(f"Public key generated. {public_key}")
            
            ssock.send(public_key.public_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PublicFormat.SubjectPublicKeyInfo
            ))
            append_log("Public key sent to server.")

            received_public_key = ssock.recv(2048)
            append_log(f"Received public key from server: {received_public_key}")
            received_public_key = serialization.load_pem_public_key(received_public_key, backend=default_backend())
            
            shared_key = derive_dh_shared_key(private_key, received_public_key)
            append_log(f"Shared key: {shared_key}")

            while True: 
                # msg_for_server = input("Enter your message for server: ")
                msg_for_server = aes_encrypt(shared_key, msg_for_server)
                ssock.send(msg_for_server)
                append_log("Message sent to server.")
                
                msg_from_server = ssock.recv(2048)
                msg_from_server = aes_decrypt(shared_key, msg_from_server)
                append_log(f"Decrypted AES message from server: {msg_from_server}")
                if msg_from_server == "exit":
                    break



In [3]:
import tkinter as tk
from tkinter import Text, Scrollbar

def append_log(message):
    log_text.config(state=tk.NORMAL)  # Set text widget to allow editing
    log_text.insert(tk.END, message + "\n")
    log_text.config(state=tk.DISABLED)  # Disable editing to make it read-only

    
    
def on_send_message():
    message = message_entry.get()
    start_client(message,append_log)
        


# Create the main window
root = tk.Tk()
root.title("CLIENT")

# # Encryption method selection
# encryption_choice = tk.StringVar()
# encryption_choice.set("Symmetric")

# encryption_label = tk.Label(root, text="Select Encryption Method:")
# encryption_label.pack()

# symmetric_radio = tk.Radiobutton(root, text="Symmetric", variable=encryption_choice, value="Symmetric")
# symmetric_radio.pack()
# asymmetric_radio = tk.Radiobutton(root, text="Asymmetric", variable=encryption_choice, value="Asymmetric")
# asymmetric_radio.pack()

# # Message input
message_label = tk.Label(root, text="Enter Message:")
message_label.pack()

message_entry = tk.Entry(root)
message_entry.pack()

# Send button
send_button = tk.Button(root, text="Send Message", command=on_send_message)
send_button.pack()

# Log display area
log_label = tk.Label(root, text="Log:")
log_label.pack()

log_text = Text(root, height=20, width=100)
log_text.pack()

scrollbar = Scrollbar(root)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

log_text.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=log_text.yview)
# Start the GUI event loop
root.mainloop()


In [4]:
if __name__ == "__main__":
   start_client("bye")
   
#openssl req -x509 -newkey rsa:4096 -keyout ca.key -out ca.crt -days 365
   """
   info for ca.crt
   PEM pass phrase: 03455
   country name: PK
   state or province name: Punjab
   locality name: Rawalpindi
   organization name: FAST
   organizational unit name: Computing
   common name: FARQ
   email address: i200621@nu.edu.pk
   """



TypeError: start_client() missing 1 required positional argument: 'append_log'